# Introduction

python3 update for libfsqca in kirq package.

In [1]:
from __future__ import division  # note that we use real division
                                 # throughout; this is primarily for
                                 # the consistency and coverage
                                 # calculations
import sys
import re
from math import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/home/gpetrini/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# General proposes QCA usage

## Errors

In [2]:
class BoundaryError(ValueError):
    """Exception raised when value is out of bounds."""
    pass

class QcaError(Exception):
    """Base class for exceptions in libfsqca module."""
    pass

class TruthTableReductionError(QcaError):
    """Exception raised when attempting to reduce a truth table that
    cannot be reduced."""
    pass

class ContradictionError(TruthTableReductionError):
    """Exception raised on attempt to reduce a truth table with a
    contradiction."""
    pass

class NoPositiveTTRowError(TruthTableReductionError):
    """Exception raised on attempt to reduce a truth table without any
    positive rows."""
    pass

class PrimeImplicantsNotFoundError(TruthTableReductionError):
    """Exception raised when unable to identify prime implicants."""
    pass

class TruthTableConstructionError(QcaError):
    """Exception raised on attempt to construct an invalid truth
    table."""
    pass

## Outcome type

In [3]:
class Contradiction(object):
    def __call__(self):
        TypeError: print("'Contradiction' object is not callable")
        raise 

    def __repr__(self):
        return 'Contradiction'

class Remainder(object):
    def __call__(self):
        TypeError: print("'Remainder' object is not callable")
        raise

    def __repr__(self):
        return 'Remainder'

class Impossible(object):
    def __call__(self):
        TypeError:print("'Impossible' object is not callable")
        raise
        
    def __repr__(self):
        return 'Impossible'


# Basic fuzzy operations

Ragin, Charles C.  2000.  Fuzzy-Set Social Science.  University of Chicago Press: Chicago.

In [4]:
def fznot(fzset): # fuzzy not
    """Boolean negation."""
    return [ 1 - x for x in fzset ]

def fzand(fzset1,fzset2): # fuzzy and
    """Boolean multiplication."""
    return [ min(x,y) for x,y in zip(fzset1,fzset2) ]

def fzor(fzset1,fzset2): # fuzzy or
    """Boolean addition."""
    return [ max(x,y) for x,y in zip(fzset1,fzset2) ]

def fzconc(fzset): # concentration
    """Boolean concentration."""
    return [ pow(x,2) for x in fzset ]

def fzdilate(fzset): # dilation
    """Boolean dilation."""
    return [ sqrt(x) for x in fzset ]


# Consistency & Coverage

Ragin, Charles C.  2006.  "Set Relations in Social Research: Evaluating Their Consistency and Coverage."  Political Analysis 14(3):291--310.

Suficient Consistency = $\frac{\sum min (x,y)}{\sum x}$

Necessary Consistency = $\frac{\sum min (x,y)}{\sum y}$

In [5]:
def consist_suf(fzset1,fzset2):
    
    sumxy = sum(fzand(fzset1, fzset2))
    sumx = sum(fzset1)

    if (sumxy == 0) and (sumx == 0):  # if both terms are zero,
        return 0                      # consistency is zero; avoid
                                      # ZeroDivision error
    else:
        return (sumxy/sumx)

def consist_nec(fzset1,fzset2):
    
    sumxy = sum(fzand(fzset1, fzset2))
    sumy = sum(fzset2)

    if (sumxy == 0) and (sumy == 0):  # if both terms are zero,
        return 0                      # consistency is zero; avoid
                                      # ZeroDivision error
    else:
        return (sumxy/sumy)

In [6]:
def is_subset(subset, superset):
    """Is one set a (proper) subset of another?

    This function is not for general use but for truth tables rows in
    which elements are True, False, or Don't Care (i.e., None).  Don't
    Cares match anything so (True,None,True) is a subset of both
    (True,True,True) or (True,False,True)."""

    for el1, el2 in zip(subset, superset):
        if (el1 is not None) and (el1 != el2):
            out = False
            break
        else:
            out = True
    return out

In [59]:
class QcaDataset:
    """QCA dataset constructor and API.

    A QcaDataset is a dataset that is suitable for QCA analysis.  
    Convert a QcaDataset instance into a truth table by passing it to
    the .from_dataset() method of a TruthTableFactory instance."""

    def __init__(self, df, outcome=[], include=[], exclude=[]):
        # df is a pandas DataFrame object. Must be fuzzyfied!
        self.outcome = outcome
        self.causal_conds = include
        self.excluded = exclude
        self.dataset = df[self.causal_conds + self.outcome] # Outcome will be the last column
        self.outcome_membership = self.dataset[self.outcome].values
        self.causal_membership = self.dataset[self.causal_conds].values
        
        for condition in self.causal_conds: # Sanity check
            if condition == self.outcome:
                QcaError: print(f"Included condition '{condition}' is the outcome")
                raise
        for exclusion in self.excluded:
            if exclusion is outcome + include:
                QcaError: (f"Excluded condition '{exclusion}' is not a causal condition")
                raise

        
        for cc in self.dataset: # validate that all data is fuzzy
            if (self.dataset[cc] < 0.0).any() or (self.dataset[cc] > 1.0).any():
                BoundaryError: print(f'Data values must be between 0.0 and 1.0. {cc} contains non-Fuzzy values')
                raise

    
    def isnec(self, causal_conds, consist_thresh):
        """
        Test if causal condition is consistent with necessity.
        ::causal_cond:: list of conditions to test
        ::consist_thresh:: float
        """
        results = []
        for condition in causal_conds:
            result = consist_nec(
                self.dataset[self.outcome].values, 
                self.dataset[condition].values) >= float(consist_thresh)
            print(f"Condition {str(condition).upper()} consistent with necessity: {result[0]}")
            results.append(result[0])
        return results

# Fuzzification

## Direct Method

In [8]:
def direct_fuzzy(inlist, lower_thresh, crossover, upper_thresh):
    """Convert an interval or ratio-level variable into a fuzzy set by
    specifying the lower threshold, crossover point, and upper
    threshold.

    Ragin, Charles C. "Fuzzy Sets: Calibration Versus Measurement."
    Forthcoming in David Collier, Henry Brady, and Janet
    Box-Steffensmeir (eds.), Methodology volume of _Oxford Handbooks
    of Political Science_"""

    try:
        deviations   = [ x - crossover for x in inlist ]
        scalar_above = 3.0/(upper_thresh - crossover)
        scalar_below = -3.0/(lower_thresh - crossover)
        scalar_at    = 0

        log_odds = []
        for deviation in deviations:
            if deviation < 0:
                log_odds.append(deviation * scalar_below)
            elif deviation == 0: 
                log_odds.append(deviation * scalar_at)
            elif deviation > 0:
                log_odds.append(deviation * scalar_above)
            
        degree_membership = [ exp(x)/(1+exp(x)) for x in log_odds ]
        return degree_membership

    except: 
        TypeError: print("non-numeric input")
        raise



# Real data example

In [56]:
df = pd.read_csv(
    '../data/International-Studies-Review-Replication-Data.csv',
    index_col="Country"
)
df.head()

,gdppc,gini,unemp,urban,youth,mobile,internet,fuel,pol,success
Country,,,,,,,,,,
Bahrain,0.84,0.42,0.58,0.89,0.26,0.68,0.89,0.58,0.11,0.0
Djibouti,0.16,0.74,1.00,0.63,0.74,0.05,0.16,0.01,0.83,0.2
Iraq,0.05,0.11,0.74,0.42,0.89,0.42,0.79,0.37,0.94,0.3
Mauritania,0.11,0.68,0.84,0.21,0.84,0.32,0.05,0.26,0.67,0.4
UAE,0.95,0.11,0.11,0.79,0.05,0.79,1.00,0.53,0.11,0.4


In [60]:
Data = QcaDataset(df=df, 
    outcome=['pol'],
    include=['gdppc', 'gini', 'unemp'], 
    exclude=['fuel'])

In [61]:
Data.isnec(causal_conds=['gini', 'unemp'], consist_thresh=0.73)

Condition GINI consistent with necessity: False
Condition UNEMP consistent with necessity: True


[False, True]

TODO: 

- [